In [ ]:
#!pip install openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df1 = pd.read_excel("C:/Users/RISHI/OneDrive/Desktop/Copper Modelling/Copper_Set.xlsx", sheet_name="Result 1")


In [ ]:
df = df1

In [ ]:
df.drop(columns=['id'])

In [ ]:
#df1.columns
numeric_conversion = ['quantity tons', 'customer', 'country','application', 'thickness', 'width','selling_price']
df[numeric_conversion] = df[numeric_conversion].apply(pd.to_numeric, errors="coerce")


In [ ]:
pd.set_option('display.max_rows',187000)
pd.set_option('display.max_columns',6000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
missing_counts = df.isna().sum()
missing_counts
missing_percentages = (missing_counts / len(df)) * 100
missing_percentages

In [ ]:
#Item_date
df = df[(df['item_date'] != 19950000)]
df = df[(df['item_date'] != 20191919)]
#df.dropna(subset = ['item_date'] , inplace = True)
df['item_date'] = pd.to_datetime(df['item_date'], format = '%Y%m%d')

In [ ]:
#Delivery_Date
#df.dropna(subset = ['delivery date'] , inplace = True)
df = df[(df['delivery date'] != 30310101)]
df = df[(df['delivery date'] != 20212222)]
df['delivery date'] = pd.to_datetime(df['delivery date'], format = '%Y%m%d')

In [ ]:
#material_ref
#df[df['material_ref'].str.contains('00000000000000000000') == True].value_counts().sum()
#df.loc[df['material_ref'].str.contains('00000000000000000000',na=False),'material_ref'] = np.NaN
df['material_ref'] = df['material_ref'].str.lstrip('0')
df['material_ref'] = df['material_ref'].apply(lambda x:'unknown' if pd.isna(x) else x )


In [ ]:
#quantity tons
df.loc[(df['quantity tons']=='e'),'quantity tons'] = np.NaN
df.loc[(df['quantity tons'] <=0) , 'quantity tons'] = np.nan

In [ ]:
#selling_price
df.loc[(df['selling_price'] <= 0) , 'selling_price'] = np.nan

In [ ]:
#country
uc = df[df['country'].isna()]
uc1 = uc['customer'].drop_duplicates()
fdf = df[df['customer'].isin(uc1.tolist())]
fdf[['customer','country']].drop_duplicates()

df.loc[(df['customer'] == 30199273.00) & (df['country'].isna()),'country'] = 27.0
df.loc[(df['customer'] == 30198586.00) & (df['country'].isna()),'country'] = 27.0
df.loc[(df['customer'] == 30196886.00) & (df['country'].isna()),'country'] = 84.0
#df['country'].fillna(df['country'].mode()[0],inplace=True)

In [ ]:
df = df.dropna()

In [ ]:
df.count()

In [ ]:
missing_counts = df.isna().sum()
missing_counts
missing_percentages = (missing_counts / len(df)) * 100
missing_percentages
missing_counts

In [ ]:
#Thickness
# Apply natural log transformation to see Distribution
df['log_thickness'] = np.log(df['thickness'])
sns.distplot(df['log_thickness'])

#Replace Missing value with Median
#thick_median = df['thickness'].median()
#df['thickness'].fillna(thick_median,inplace=True)
#df[df['thickness'].isna()]

#Again Apply log transformation to see Distribution
#df['log_thickness'] = np.log(df['thickness'])

#Remove Outlier based on the Lower and Upper Whisker
# Calculate quartiles on log-transformed data
Q1_log = df['log_thickness'].quantile(0.25)
Q3_log = df['log_thickness'].quantile(0.75)

# Calculate IQR
IQR_log = Q3_log - Q1_log

# Calculate whiskers
lower_whisker_log = Q1_log - 1.5 * IQR_log
upper_whisker_log = Q3_log + 1.5 * IQR_log

# Find outliers in log-transformed data
lower_outliers_log = df[df['log_thickness'] < lower_whisker_log]
upper_outliers_log = df[df['log_thickness'] > upper_whisker_log]

print(f'Q1 (Log-transformed): {Q1_log}')
print(f'Q3 (Log-transformed): {Q3_log}')
print(f'IQR (Log-transformed): {IQR_log}')
print(f'Lower Whisker (Log-transformed): {lower_whisker_log}')
print(f'Upper Whisker (Log-transformed): {upper_whisker_log}')

print('\nLower Outliers (Log-transformed):')
print(lower_outliers_log)

print('\nUpper Outliers (Log-transformed):')
print(upper_outliers_log)

df = df[(df['log_thickness'] >= lower_whisker_log) & (df['log_thickness'] <= upper_whisker_log)]
plt.figure(figsize=(22, 18))
sns.distplot( df['log_thickness'])
plt.figure(figsize=(22, 18))
sns.boxplot( x='log_thickness',data=df)
plt.show()


In [ ]:
#Quantity tons
df['log_quantity'] = np.log(df['quantity tons'])
plt.figure(figsize=(22, 18))
sns.distplot( df['log_quantity'])
df['log_quantity'].skew() #slightly negatively skewed but the outliers are acceptable
Q1 = df['log_quantity'].quantile(0.25)
Q3 = df['log_quantity'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold_low = Q1 - 1.5 * IQR
outlier_threshold_high = Q3 + 1.5 * IQR
# Identify outliers
outliers = df[(df['log_quantity'] < outlier_threshold_low) | (df['log_quantity'] > outlier_threshold_high)]
outliers.head()

In [ ]:
#selling_price
df['log_selling_price'] = np.log(df['selling_price'])
plt.figure(figsize=(22, 18))
sns.boxplot( x='log_selling_price',y = 'item type',data=df)
plt.show()
sns.distplot(df['log_selling_price'])
plt.show()
Q1 = df['log_selling_price'].quantile(0.25)
Q3 = df['log_selling_price'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold_low = Q1 - 1.5 * IQR
outlier_threshold_high = Q3 + 1.5 * IQR
# Identify outliers
outliers = df[(df['log_selling_price'] < outlier_threshold_low) | (df['log_selling_price'] > outlier_threshold_high)]
outliers.count()

In [ ]:
sns.distplot(df['width']) #skewness almost equal to Zero
plt.show()
sns.distplot(df['log_selling_price'])
plt.show()
sns.distplot(df['log_quantity'])
plt.show()
sns.distplot(df['log_thickness'])
plt.show()

In [ ]:
c=['width','log_selling_price','log_quantity','log_thickness']
print(df[c].skew())


In [ ]:
#!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [ ]:
status=df['status'].unique()
item=df['item type'].unique()

print (status ,"," ,item)

encode=LabelEncoder()
df['status_encoded'] = encode.fit_transform(df['status'])

encode=LabelEncoder()
df['item_encoded'] = encode.fit_transform(df['item type'])

status=df['status_encoded'].unique()
item=df['item_encoded'].unique()
print (status ,"," ,item)

In [ ]:
import pickle
status=df['status'].unique()
item=df['item type'].unique()

encode=LabelEncoder()
encoded_status = encode.fit_transform(status)

encode=LabelEncoder()
encoded_item_type = encode.fit_transform(item)


with open('status.pkl', 'wb') as file:
    pickle.dump(encoded_status, file)
with open('item_type.pkl', 'wb') as file:
    pickle.dump(encoded_item_type, file)

with open('status.pkl', 'rb') as file:
    encoded_status = pickle.load(file)
with open('item_type.pkl', 'rb') as file:
    encoded_item_type = pickle.load(file)

In [ ]:
corr_df = df[['log_quantity','status_encoded','item_encoded','application','log_thickness','width','country','customer','product_ref']]
corr_df.corr()
#sns.heatmap(corr_df, annot = True,cmap = "viridis")

In [ ]:
df.columns

In [ ]:
X=df[['log_quantity','status_encoded','item_encoded','application','log_thickness','width','country','product_ref']]
y=df[['log_selling_price']].values

In [ ]:
scaler = StandardScaler().fit(X)
#X = scaler.transform(X)

In [ ]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
with open('scaler.pkl', 'rb') as file:
    scaled_data = pickle.load(file)    
pred_model= scaled_data.transform(X)


In [ ]:
print(len(X), len(y)) 
print(X.shape)  # Should be (n_samples, n_features)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

x_train,x_test,y_train,y_test = train_test_split(pred_model,y,test_size=0.3,random_state=42)
dtreg = DecisionTreeRegressor(random_state=42)
dtreg.fit(x_train,y_train)
y_predict = dtreg.predict((x_test))
r_scr = metrics.r2_score(y_test,y_predict)
print(r_scr)
print(metrics.mean_squared_error(y_test,y_predict))
print(np.sqrt(metrics.mean_squared_error(y_test,y_predict)))
print(metrics.mean_absolute_error(y_test,y_predict)) #for outlier in the data use MAE


In [ ]:
param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dtreg = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=dtreg, param_grid=param_grid, 
                           cv=5, )
grid_search.fit(x_train,y_train)
best_dtree_reg = grid_search.best_estimator_
y_predict = best_dtree_reg.predict((x_test))
best_params = grid_search.best_params_
print(metrics.mean_squared_error(y_test,y_predict))
print(np.sqrt(metrics.mean_squared_error(y_test,y_predict)))
print(metrics.mean_absolute_error(y_test,y_predict))
mse = metrics.mean_squared_error(y_test, y_predict)
rmse = mse ** 0.5
print(f"Best parameters: {best_params}")
print(f"Best estimator: {best_dtree_reg}")
print(f"Test RMSE: {rmse}")

In [ ]:
r_scr = metrics.r2_score(y_test,y_predict)
print(r_scr)

In [ ]:
import pickle
with open('regression.pkl','wb') as file:
    pickle.dump(dtreg,file)

In [ ]:
with open('regression.pkl','rb') as file:
    reg = pickle.load(file)

In [ ]:
prediction = reg.predict([[4.02,7,3,10.00,1.79,2000.00,40.00,30225641.00,640405]])
prediction

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classfier_df = df[(df['status'] == 'Won') | (df['status'] == 'Lost') ]

In [ ]:
status=classfier_df['status'].unique()
item=classfier_df['item type'].unique()

print (status ,"," ,item)

encode=LabelEncoder()
classfier_df['status_encoded'] = encode.fit_transform(classfier_df['status'])

encode=LabelEncoder()
classfier_df['item_encoded'] = encode.fit_transform(classfier_df['item type'])

status=classfier_df['status_encoded'].unique()
item=classfier_df['item_encoded'].unique()
print (status ,"," ,item)

In [ ]:
X = classfier_df[['log_quantity','log_selling_price','item_encoded','application','log_thickness','width','country','product_ref']]
y = classfier_df['status_encoded']


In [ ]:
scaler_classify = StandardScaler().fit(X)
X = scaler_classify.transform(X)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
clf = DecisionTreeClassifier(criterion="entropy", max_depth=20)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:

with open('scaling_classify.pkl', 'wb') as file:
    pickle.dump(scaler_classify, file)

In [ ]:
with open("Decission_tree_classification.pkl", 'wb') as file:
    pickle.dump(clf, file)